In [1]:
import os
import numpy as np
import pandas as pd
from recbole.quick_start import run_recbole

C:\Users\29700\anaconda3\envs\modeltest\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-22 11:16:02,508	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-22 11:16:02,809	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [7]:
import time
import datetime

[sequential recommendation parameter guidance](https://recbole.io/docs/get_started/started/sequential.html)



In [61]:
path = "G:/My Drive/2021/Bias/sumo_simulation/"
os.chdir(path)

#get obs data matrix
data_path_lst = []
for i in os.listdir():
    if len(i) == 12 and '.csv' in i:
        data_path_lst.append(i)
data_path_lst

['20170404.csv',
 '20170405.csv',
 '20170406.csv',
 '20170411.csv',
 '20170412.csv',
 '20170413.csv',
 '20170418.csv',
 '20170419.csv',
 '20170420.csv',
 '20170426.csv',
 '20170427.csv',
 '20170425.csv']

In [ ]:
#reference source: https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

In [3]:
#convet data to recbole data format
test_dt = pd.read_csv(data_path_lst[0])

#convert time info to timestamp
timestep_convert = lambda x: time.mktime(datetime.datetime.strptime(('2017_4_'+x[:-4]),
                                                                    "%Y_%m_%d_%H_%M").timetuple())
test_dt['date_time'] = test_dt['key'].apply(timestep_convert)

In [46]:
test_dt[]

,newid,sum,key,taz,date,time,date_time
0,26,1,4_3_55_608,608,4,3:55,1.491303e+09
1,26,2,4_4_5_615,615,4,4:05,1.491304e+09
2,26,1,4_3_40_578,578,4,3:40,1.491302e+09


In [52]:
temp = test_dt[['newid','taz','date_time']].rename(columns={'newid': 'user_id:token', 
                                                     'taz': 'item_id:token', 
                                                     'date_time': 'timestamp:float'})

In [58]:
temp.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

,user_id:token,item_id:token,timestamp:float
0,26,608,1.491303e+09
1,26,615,1.491304e+09
2,26,578,1.491302e+09
3,26,627,1.491302e+09
4,32,502,1.491331e+09
...,...,...,...
227191,458481,567,1.491372e+09
227192,458481,566,1.491372e+09
227193,458481,431,1.491372e+09
227194,458481,550,1.491373e+09


In [63]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

In [68]:
os.listdir('../SUMO_simulation/')

['MatSumo-main.zip',
 'MatSumo-main',
 'obsmx04042017.csv',
 'obsmx04052017.csv',
 'obsmx04062017.csv',
 'obsmx04112017.csv',
 'obsmx04122017.csv',
 'obsmx04132017.csv',
 'obsmx04182017.csv',
 'obsmx04192017.csv',
 'obsmx04202017.csv',
 'obsmx04252017.csv',
 'obsmx04262017.csv',
 'obsmx04272017.csv',
 'newid_file.txt',
 'newid_tsid_aggregation.txt',
 'MatSim_userguide.pdf',
 'obs_matrix_sum.csv',
 'Seattle_network.osm.pbf',
 'matsim-example-project',
 'matsim-episim-example-project',
 'matsim-code-examples',
 'model_results.csv',
 'svd.txt',
 'model_results_nmf_larger10.csv',
 'model_results_nmf_larger5.csv',
 'indi_meeting.pptx',
 '~$sumo_sim.pptx',
 'obs_matrixbyday.csv',
 'WH_obs_2000601.csv',
 'nmf_est_result_0405obs.csv',
 'obs_byday_model_results.csv',
 'modelpredresults78010103.csv',
 'modelpredresults78010100.csv',
 'laltontaz04042017.csv',
 'lalton04042017.csv',
 'latlon04042017.csv',
 'latlontaz04042017.csv',
 'latlon04052017.csv',
 'latlontaz04052017.csv',
 'latlon04062017.c

In [78]:
parameter_dict = {
    'data_path': '../SUMO_simulation/',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'TEM_LIST_LENGTH_FIELD':2,
    'neg_sampling':None,
    'train_neg_sample_args': None,
    'epochs': 50
}

config =  Config(model='GRU4Rec', dataset='recbox_data', config_dict=parameter_dict)

In [79]:
init_seed(config['seed'], config['reproducibility'])

In [80]:
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config)

22 Apr 16:57    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ../SUMO_simulation/recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset 

In [82]:
dataset = create_dataset(config)

In [83]:
logger.info(dataset)

22 Apr 16:57    INFO  recbox_data
The number of users: 13710
Average actions of users: 16.57276241884893
The number of items: 168
Average actions of items: 1360.4550898203593
The number of inters: 227196
The sparsity of the dataset: 90.13597999374805%
Remain Fields: ['user_id', 'item_id', 'timestamp']
recbox_data
The number of users: 13710
Average actions of users: 16.57276241884893
The number of items: 168
Average actions of items: 1360.4550898203593
The number of inters: 227196
The sparsity of the dataset: 90.13597999374805%
Remain Fields: ['user_id', 'item_id', 'timestamp']


In [84]:
train_data, valid_data, test_data = data_preparation(config, dataset)

22 Apr 16:58    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
22 Apr 16:58    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]


In [85]:
# model loading and initialization
model = GRU4Rec(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

22 Apr 16:58    INFO  GRU4Rec(
  (item_embedding): Embedding(168, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 92736
GRU4Rec(
  (item_embedding): Embedding(168, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 92736
22 Apr 17:02    INFO  epoch 0 training [time: 248.45s, train loss: 379.8291]
epoch 0 training [time: 248.45s, train loss: 379.8291]
22 Apr 17:02    INFO  Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
22 Apr 17:06    INFO  epoch 1 training [time: 253.95s, train loss: 236.9083]
epoch 1 training [time: 253.95s, train loss: 23

22 Apr 21:25    INFO  Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
22 Apr 21:29    INFO  epoch 29 training [time: 274.14s, train loss: 189.6485]
epoch 29 training [time: 274.14s, train loss: 189.6485]
22 Apr 21:29    INFO  Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
22 Apr 21:34    INFO  epoch 30 training [time: 265.64s, train loss: 189.4596]
epoch 30 training [time: 265.64s, train loss: 189.4596]
22 Apr 21:34    INFO  Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
22 Apr 21:39    INFO  epoch 31 training [time: 306.44s, train loss: 188.9732]
epoch 31 training [time: 306.44s, train loss: 188.9732]
22 Apr 21:39    INFO  Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
Saving current: saved\GRU4Rec-Apr-22-2024_16-58-24.pth
22 Apr 21:45    INFO  epoch 32 training [time: 340.13s, train loss: 18

In [90]:
best_valid_score

-inf

In [89]:
best_valid_result

In [86]:
from recbole.utils.case_study import full_sort_topk
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]#fist element in array is 'PAD'(default of Recbole) ->remove it 

In [87]:
topk_items = []
for internal_user_id in list(range(dataset.user_num))[1:]:
    _, topk_iid_list = full_sort_topk([internal_user_id], model, test_data, k=12, device=config['device'])
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)
print(len(topk_items))

IndexError: index -1 is out of bounds for dimension 0 with size 0

In [88]:
external_item_str = [' '.join(x) for x in topk_items]
result = pd.DataFrame(external_user_ids, columns=['customer_id'])
result['prediction'] = external_item_str
result.head()

ValueError: Length of values (3) does not match length of index (13709)